# Imports

In [66]:
from __future__ import print_function
import requests
import numpy as np
import pprint
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from matplotlib import dates
from matplotlib.pyplot import figure
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from dateutil.parser import parse
import locale
import plotly.express as px
import plotly.graph_objects as go

# Locale for setting the number format

In [67]:
locale.setlocale(locale.LC_ALL, 'en_US')

'en_US'

# Endpoint setting and getting country-slug list for populating dropdown

In [68]:
url_countries = "https://api.covid19api.com/countries"
countries = requests.get(url_countries) #countries.status_code Assuming status code is 200
countries = countries.json()
country_slug_list = []
country_list = []
for country in countries:
    country_slug_list.append(country['Slug'])
for country in countries:
    country_list.append(country['Country'])
country_slug_list.sort()
country_list.sort() 

# Dropdown to select country

In [52]:
dropdown_country = widgets.Dropdown(
    options=(country_slug_list),
#     value='Country',
    description='Country:',
    disabled=False,
)
display(dropdown_country)

Dropdown(description='Country:', options=('afghanistan', 'ala-aland-islands', 'albania', 'algeria', 'american-…

In [53]:
country_name = dropdown_country.value
country_name

'afghanistan'

# Endpoint setting and populating lists

In [54]:
url_complete_status = f"https://api.covid19api.com/total/dayone/country/{country_name}"
stats_since_day_one = requests.get(url_complete_status)
stats_since_day_one = stats_since_day_one.json()
active_list = []
confirmed_list =[]
deaths_list = []
recovered_list = []
date_list = []
new_date_list = []

for stat in stats_since_day_one:
    active_list.append(stat['Active'])
    confirmed_list.append(stat['Confirmed'])
    deaths_list.append(stat['Deaths'])
    recovered_list.append(stat['Recovered'])
    date_list.append(stat['Date'])

# Making Date ready for further plotting and calculations

In [55]:
for date in date_list:
    new_date_list.append(date[:10])

date_objects = [datetime.strptime(date, '%Y-%m-%d').date() for date in new_date_list]  

# Capitalizing first letter of country name for better representation

In [56]:
country_name_capitalized = country_name.capitalize()
country_name_capitalized_spaced = country_name_capitalized.replace('-',' ')
country_name_final_representation = country_name_capitalized_spaced.title()

# Function for finding max one day increase

In [57]:
def max_one_day_increase(empty_list,data_list):
    for i in range(1,len(data_list)):
        empty_list.append(data_list[i]-data_list[(i-1)])
    return max(empty_list)

# Function for statistics in past 24 hours

In [58]:
def change_past_24_hours(data_list):
    return (data_list[-1]-data_list[-2])


# All calculations

In [59]:
total_confirmed_cases = confirmed_list[-1]
total_active_cases = active_list[-1]
total_deaths = deaths_list[-1]
total_recovered = recovered_list[-1]
percent_active = round(total_active_cases/total_confirmed_cases*100,2)
percent_death = round(total_deaths/total_confirmed_cases*100,2)
percent_recovered = round(total_recovered/total_confirmed_cases*100,2)

# days calculation
first_case_reported = new_date_list[0]
date_list_calculate = [datetime.strptime(dateInDatetime, '%Y-%m-%d') for dateInDatetime in new_date_list]
date_first_case = date_list_calculate[0]
current_date = datetime.now()
days_since_first_case = (current_date - date_first_case).days

# Average calculation
avg_cases_per_day = round(total_confirmed_cases/days_since_first_case)
avg_deaths_per_day = round(total_deaths/days_since_first_case)
avg_recoveries_per_day = round(total_recovered/days_since_first_case)

# 2 weeks calculation
date_before_14_days = current_date - timedelta(days=14)
year = date_before_14_days.strftime("%Y")
month = date_before_14_days.strftime("%m")
day = date_before_14_days.strftime("%d")

date_before_14_days_string = f"{year}-{month}-{day}"
date_before_14_days_string

index_before_14_days = new_date_list.index(date_before_14_days_string)
confirmed_cases_14_days = total_confirmed_cases - confirmed_list[index_before_14_days]
active_cases_14_days = total_active_cases - active_list[index_before_14_days]
deaths_14_days = total_deaths - deaths_list[index_before_14_days]
recovered_14_days = total_recovered - recovered_list[index_before_14_days]
avg_cases_per_day_14_days = round(confirmed_cases_14_days/14)
avg_deaths_per_day_14_days = round(deaths_14_days/14)
avg_recoveries_per_day_14_days = round(recovered_14_days/14)

#Max One Day increase
one_day_increase_confirmed_cases = []
max_one_day_increase_confirmed_cases = max_one_day_increase(one_day_increase_confirmed_cases,confirmed_list)
one_day_increase_deaths = []
max_one_day_increase_deaths = max_one_day_increase(one_day_increase_deaths,deaths_list)
one_day_increase_recovered = []
max_one_day_increase_recovered = max_one_day_increase(one_day_increase_recovered,recovered_list)


#Past 24 hours
past_24_hours_confirmed_cases = change_past_24_hours(confirmed_list)
past_24_hours_active_cases = change_past_24_hours(active_list)
past_24_hours_deaths = change_past_24_hours(deaths_list)
past_24_hours_recovered = change_past_24_hours(recovered_list)

# Function for dynamically plotting graphs using plotly

In [60]:

def yLabelTitle(fig,y):
    if y == confirmed_list:
        fig.update_layout(
        title={
        'text': f"COVID-19 Confirmed Cases in {country_name_final_representation} || Total Confirmed Cases - {total_confirmed_cases:n}  ",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        xaxis_title="Date",
        yaxis_title="Confirmed Cases",
        hovermode='x'
        )
    elif y == active_list:
        fig.update_layout(
        title={
        'text': f"COVID-19 Active Cases in {country_name_final_representation} || Total Active Cases - {total_active_cases:n} || Active - {percent_active}%",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        xaxis_title="Date",
        yaxis_title="Active Cases",
        hovermode='x'
        )
    elif y == deaths_list:
         fig.update_layout(
        title={
        'text': f"COVID-19 Deaths in {country_name_final_representation} || Total Deaths - {total_deaths:n} || Deaths - {percent_death}% ",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        xaxis_title="Date",
        yaxis_title="Total Deaths",
        hovermode='x'
        )
    elif y == recovered_list:
        fig.update_layout(
        title={
        'text': f"COVID-19 Recoveries in {country_name_final_representation} || Total Recovered - {total_recovered:n} || Recovered - {percent_recovered}%  ",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        xaxis_title="Date",
        yaxis_title="Total Recovered",
        hovermode='x'
        )
    else:
        print("No Plot To Show")

# Total Confirmed Cases

In [61]:
x = date_objects
y = confirmed_list
fig = go.Figure(data=go.Scatter(x=x, y=y))
yLabelTitle(fig,y)
fig.show()

# Total Active Cases

In [62]:
x = date_objects
y = active_list
fig = go.Figure(data=go.Scatter(x=x, y=y))
yLabelTitle(fig,y)
fig.show()

# Total Deaths

In [63]:
x = date_objects
y = deaths_list
fig = go.Figure(data=go.Scatter(x=x, y=y))
yLabelTitle(fig,y)
fig.show()

# Total Recoveries

In [64]:
x = date_objects
y = recovered_list
fig = go.Figure(data=go.Scatter(x=x, y=y))
yLabelTitle(fig,y)
fig.show()

# Relevant Statistics

In [65]:
past_24_hours_confirmed_cases = change_past_24_hours(confirmed_list)
past_24_hours_active_cases = change_past_24_hours(active_list)
past_24_hours_deaths = change_past_24_hours(deaths_list)
past_24_hours_recovered = change_past_24_hours(recovered_list)

print(f"Statistics for {country_name_final_representation} For Last 24 hours (1 day)")
print(f"Confirmed Cases : {past_24_hours_confirmed_cases:n}")
if past_24_hours_active_cases < 0:
    print(f"Active Cases : {abs(past_24_hours_active_cases):n} Decrease")
elif active_cases_14_days > 0:
    print(f"Active Cases : {abs(past_24_hours_active_cases):n} Increase")
else:
    print(f"No Active Cases")
print(f"Deaths : {past_24_hours_deaths:n}")
print(f"Recovered : {past_24_hours_recovered:n}")

print("\n")

print(f"Statistics for {country_name_final_representation} For Last 14 days (2 weeks)")
print(f"Confirmed Cases : {confirmed_cases_14_days:n}")
if active_cases_14_days < 0:
    print(f"Active Cases : {abs(active_cases_14_days):n} Decrease")
elif active_cases_14_days > 0:
    print(f"Active Cases : {abs(active_cases_14_days):n} Increase")
else:
    print(f"No Active Cases")
print(f"Deaths : {deaths_14_days:n}")
print(f"Recovered : {recovered_14_days:n}")
print(f"Average Cases Per Day : {avg_cases_per_day_14_days:n}")
print(f"Average Deaths Per Day : {avg_deaths_per_day_14_days:n}")
print(f"Average Recoveries Per Day : {avg_recoveries_per_day_14_days:n}")

print("\n")

print(f"All Time Statistics for {country_name_final_representation}")
print(f"Total Confirmed Cases : {total_confirmed_cases:n}")
print(f"Total Active Cases : {total_active_cases:n}")
print(f"Total Deaths : {total_deaths:n}")
print(f"Total Recovered : {total_recovered:n}")
print(f"Active Percentage : {percent_active:n}%")
print(f"Death Percentage : {percent_death}%")
print(f"Recovery Percentage : {percent_recovered}%")
print(f"First Case Reported : {first_case_reported}")
print(f"Days since First Case : {days_since_first_case}")
print(f"Average Cases Per Day : {avg_cases_per_day:n}")
print(f"Average Deaths Per Day : {avg_deaths_per_day:n}")
print(f"Average Recoveries Per Day : {avg_recoveries_per_day:n}")
print(f"Maximum one day increase in confirmed cases : {max_one_day_increase_confirmed_cases:n}")
print(f"Maximum one day increase in deaths : {max_one_day_increase_deaths:n}")
print(f"Maximum one day increase in recoveries : {max_one_day_increase_recovered:n}")

Statistics for Afghanistan For Last 24 hours (1 day)
Confirmed Cases : 625
Active Cases : 608 Increase
Deaths : 7
Recovered : 10


Statistics for Afghanistan For Last 14 days (2 weeks)
Confirmed Cases : 6,817
Active Cases : 6,279 Increase
Deaths : 91
Recovered : 447
Average Cases Per Day : 487
Average Deaths Per Day : 6
Average Recoveries Per Day : 32


All Time Statistics for Afghanistan
Total Confirmed Cases : 12,456
Total Active Cases : 11,091
Total Deaths : 227
Total Recovered : 1,138
Active Percentage : 89.04%
Death Percentage : 1.82%
Recovery Percentage : 9.14%
First Case Reported : 2020-02-24
Days since First Case : 94
Average Cases Per Day : 133
Average Deaths Per Day : 2
Average Recoveries Per Day : 12
Maximum one day increase in confirmed cases : 782
Maximum one day increase in deaths : 17
Maximum one day increase in recoveries : 80


36188

36188